In [2]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.preprocessing import LabelEncoder 

In [3]:
#Importamos los set de datos.
eventos = pd.read_csv("../events_up_to_01062018.csv", low_memory=False)
train = pd.read_csv("../trocafone_kaggle_test.csv")

In [4]:
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [5]:
#Creamos un dataframe de features
features = train

In [6]:
features.shape

(19415, 1)

In [7]:
#Feature: Es un usuario que retorno alguna vez?

df = eventos.groupby(['person'])['new_vs_returning'].agg('count').reset_index()
df['retornoAlSitio'] = df["new_vs_returning"].apply(lambda x: True if x>0 else False)
del df['new_vs_returning']
features = pd.merge(features, df, on='person', how= 'left')


In [8]:
#Feature: Cantidad de tipo de estado de dispositivo que busco.

df = eventos.groupby('person')['condition'].value_counts().unstack()
df = df.rename(columns={'Bom': 'cantidad_visualizaciones_buen_estado', 
                 'Bom - Sem Touch ID': 'cantidad_visualizaciones_buen_estado_sin_touch', 
                 'Novo': 'cantidad_visualizaciones_nuevo',
                 'Muito Bom': 'cantidad_visualizaciones_muy_buen_estado',
                 'Excelente': 'cantidad_visualizaciones_excelente'})

df = df.fillna(0)
features = pd.merge(features, df, on='person', how= 'left')

In [9]:
#Feature: Cantidad de eventos generados totales

df = eventos.groupby(['person'])['event'].agg('count').to_frame('cantidad_eventos_generados').reset_index()
features = pd.merge(features, df, on='person', how= 'left')


In [10]:
#Features: Cantidad por tipo de evento generado
df = eventos.groupby('person')['event'].value_counts().unstack()
df = df.fillna(0)
df = df.rename(columns={'ad campaign hit': 'cantidadIngresosPorCampania', 
                 'brand listing': 'cantidadDeVisualizacionesPorMarca', 
                 'checkout': 'cantidadDeCheckouts',
                 'conversion': 'cantidadConversiones',
                 'generic listing': 'cantidadDeVisualizacionesDeLaHome',
                 'lead': 'cantidadDeGeneracionDeNotificaciones',
                 'search engine hit': 'cantidadDeIngresosPorBuscador',
                 'searched products': 'cantidadDeProductosBuscados',
                 'staticpage': 'cantidadDeVisitas',
                 'viewed product': 'cantidadDeVisualizacionesDeProducto',
                 'visited site': 'cantidadDeIngresosPorUrl',
                })
features = pd.merge(features, df, on='person', how= 'left')

In [11]:
#Features: Cantidad de visualizaciones por marca
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos_con_marca['model'].str.split(' ').str[0]
df = eventos_con_marca.groupby('person')['marca'].value_counts().unstack()
df = df.fillna(0)
df = df.rename(columns={'Asus': 'cantidadDeVisualizacionesAsus', 
                 'LG': 'cantidadVisualizacionesLG', 
                 'Lenovo': 'cantidadVisualizacionesLenovo',
                 'Outros': 'cantidadVisualizacionesOtros',
                 'Quantum': 'cantidadVisualizacionesQuantum',
                 'Samsung': 'cantidadVisualizacionesSamsung',
                 'Sony': 'cantidadVisualizacionesSony',
                 'Xiaomi': 'cantidadVisualizacionesXiaomi',
                 'iPad': 'cantidadVisualizacionesiPad',
                 'iPhone': 'cantidadDeVisualizacionesiPhone'
                })
features = pd.merge(features, df, on='person', how= 'left')

In [12]:
#Features: Region del usuario
df = eventos.loc[eventos.event.str.contains('visited site'),]
df = df.groupby(['person']).agg({'region': 'first'})
df = df.fillna('NA')
features = pd.merge(features, df, on='person', how= 'left')

In [13]:
#Features: Ciudad del usuario
df = eventos.loc[eventos.event.str.contains('visited site'),]
df = df.groupby(['person']).agg({'city': 'first'})
df = df.fillna('NA')
features = pd.merge(features, df, on='person', how= 'left')

In [14]:
#Features: Pais del usuario
df = eventos.loc[eventos.event.str.contains('visited site'),]
df = df.groupby(['person']).agg({'country': 'first'})
df = df.fillna('NA')
features = pd.merge(features, df, on='person', how= 'left')

In [15]:
#Features: Region del usuario
df = eventos.loc[eventos.event.str.contains('visited site'),]
df = df.groupby(['person']).agg({'device_type': 'first'})
df = df.fillna('NA')
features = pd.merge(features, df, on='person', how= 'left')

In [16]:
#Features: Cantidad por tipo de canal proveniente  generado
df = eventos.groupby('person')['channel'].value_counts().unstack()
df = df.fillna(0)
df = df.rename(columns={'Direct': 'cantidadDeIngresosPorCanalDirecto', 
                 'Email': 'cantidadDeIngresosPorCanalEmail', 
                 'Organic': 'cantidadDeIngresosPorCanalOrganico',
                 'Paid': 'cantidadDeIngresosPorCanalOrganicoPago',
                 'Referral': 'cantidadDeIngresosPorCanalReferido',
                 'Social': 'cantidadDeIngresosPorCanalRedSocial',
                 'Unknown': 'cantidadDeIngresosPorCanalDesconocido'
                })
features = pd.merge(features, df, on='person', how= 'left')

In [17]:
#Features: Cantidad por tipo de campania proveniente  generado
df = eventos.groupby('person')['campaign_source'].value_counts().unstack()
df = df.fillna(0)
features = pd.merge(features, df, on='person', how= 'left')


In [18]:
#Encodeamos las features que nos numeros
lb_make = LabelEncoder()

features['device_type'] = lb_make.fit_transform(features['device_type'].astype(str))
features['city'] = lb_make.fit_transform(features['city'].astype(str))
features['region'] = lb_make.fit_transform(features['region'].astype(str))
features['country'] = lb_make.fit_transform(features['country'].astype(str))

In [19]:
features = features.fillna(0)

In [20]:
features.to_csv("../set_a_predecir.csv")